In [14]:
import pyodbc
import pandas as pd
import json
import socket
from pathlib import Path

# Database connection parameters
server = "demo01"
database = "demo01_KahuaDataStore_11610510910911783"
username = "demo01_kahuaDataStoreUser_89122113112107117"
password = "Playful3"

# First, test basic network connectivity
print("🔍 Diagnosing connection...")
try:
    ip = socket.gethostbyname(server)
    print(f"✅ DNS resolved: {server} -> {ip}")
except socket.gaierror:
    print(f"❌ DNS lookup failed for {server}")

# Test if port 1433 is reachable
try:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(5)
    result = sock.connect_ex((server, 1433))
    sock.close()
    if result == 0:
        print(f"✅ Port 1433 is open on {server}")
    else:
        print(f"❌ Port 1433 is NOT reachable on {server}")
except Exception as e:
    print(f"❌ Port check failed: {e}")

# Try to connect to the database (using connection params instead of string to handle special chars)
conn = None
try:
    print("\n🔌 Attempting database connection...")
    # Use braces around password to handle special characters like @
    connection_string = (
        f"DRIVER={{ODBC Driver 17 for SQL Server}};"
        f"SERVER={server};"
        f"DATABASE={database};"
        f"UID={username};"
        f"PWD={{{password}}};"  # Extra braces to escape special chars
        f"Connection Timeout=10"
    )
    conn = pyodbc.connect(connection_string)
    print("✅ Successfully connected to the database!")
    
    # Get list of tables
    query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'"
    tables = pd.read_sql(query, conn)
    print(f"\n📊 Available tables in the DataStore ({len(tables)} tables):")
    print(tables)
except pyodbc.Error as e:
    print(f"❌ Database connection failed: {e}")

🔍 Diagnosing connection...
✅ DNS resolved: demo01 -> 10.10.12.181
✅ Port 1433 is open on demo01

🔌 Attempting database connection...
✅ Successfully connected to the database!


C:\Users\AWright\AppData\Local\Temp\ipykernel_34784\2411194386.py:52: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables = pd.read_sql(query, conn)



📊 Available tables in the DataStore (1889 tables):
                                             TABLE_NAME
0     centrenium_CloseoutPackage_CloseoutPackageRevi...
1                  kahua_SecureSignature_EnvelopeStatus
2     runding_PerformanceTask_PerformanceTask_Perfor...
3     kahua_AssetManager_Assets_PhotovalicSystemAsse...
4     kahua_Sustainability_ScopeType_HVAC_BoilersorH...
...                                                 ...
1884                ezApps_Checklist_Checklist_Sections
1885              kahua_AEC_BidSolicitations_BidCompany
1886       runding_kahua_scheduleTask_Task_Predecessors
1887                 kahua_Core_kahua_FileData_Comments
1888  kahua_AssetManager_Types_GlassType_DataScope_P...

[1889 rows x 1 columns]


In [16]:
# Search for tables by keyword
def search_tables(keyword):
    """Find tables containing a keyword"""
    matches = tables[tables['TABLE_NAME'].str.contains(keyword, case=False)]
    print(f"Found {len(matches)} tables matching '{keyword}':")
    return matches

# Common entity types to explore
print("🔍 Quick table searches:")
print(f"\nContracts: {len(search_tables('Contract').head(0))} tables")
print(f"RFIs: {len(search_tables('RFI').head(0))} tables")
print(f"Submittals: {len(search_tables('Submittal').head(0))} tables")
print(f"Punch Lists: {len(search_tables('Punch').head(0))} tables")
print(f"Change Orders: {len(search_tables('ChangeOrder').head(0))} tables")

🔍 Quick table searches:
Found 79 tables matching 'Contract':

Contracts: 0 tables
Found 19 tables matching 'RFI':
RFIs: 0 tables
Found 45 tables matching 'Submittal':
Submittals: 0 tables
Found 14 tables matching 'Punch':
Punch Lists: 0 tables
Found 35 tables matching 'ChangeOrder':
Change Orders: 0 tables


In [19]:
# View Contract tables
contract_tables = search_tables('Contract')
contract_tables.head(20)

Found 79 tables matching 'Contract':


,TABLE_NAME
7,kahua_ClientContractChangeOrder_ClientContract...
36,kahua_ClientContract_ClientContract
57,kahua_ClientContractChangeOrder_ClientContract...
75,kahua_ClientContract_ClientContract_Comments
122,kahua_ClientContract_ClientContract_Items
398,kahua_ClientContractInvoice_ClientContractInvoice
443,kahua_ClientContractInvoice_ClientContractInvo...
479,kahua_ContractChangeOrder_ContractChangeOrderItem
490,kahua_ClientContractInvoice_ClientContractInvo...
522,kahua_ContractChangeOrder_ContractChangeOrderI...


In [20]:
# Helper to preview a table's structure and sample data
def preview_table(table_name, limit=5):
    """Preview a table's columns and sample rows"""
    print(f"\n📋 Table: {table_name}")
    print("="*60)
    
    # Get column info
    col_query = f"""
    SELECT COLUMN_NAME, DATA_TYPE, IS_NULLABLE 
    FROM INFORMATION_SCHEMA.COLUMNS 
    WHERE TABLE_NAME = '{table_name}'
    ORDER BY ORDINAL_POSITION
    """
    columns = pd.read_sql(col_query, conn)
    print(f"\n📐 Columns ({len(columns)}):")
    print(columns.to_string())
    
    # Get sample data
    try:
        sample_query = f"SELECT TOP {limit} * FROM [{table_name}]"
        sample = pd.read_sql(sample_query, conn)
        print(f"\n📊 Sample data ({len(sample)} rows):")
        return sample
    except Exception as e:
        print(f"⚠️ Could not fetch sample: {e}")
        return columns

In [21]:
# Preview the main Contract table
preview_table('kahua_Contract_Contract')


📋 Table: kahua_Contract_Contract

📐 Columns (95):
                            COLUMN_NAME DATA_TYPE IS_NULLABLE
0                                    Id    bigint          NO
1                     DomainPartitionId    bigint         YES
2                           PartitionId    bigint         YES
3                       CreatedDateTime  datetime         YES
4                      ModifiedDateTime  datetime         YES
5                         EntityLinkUrl  nvarchar         YES
6                               Addenda  nvarchar         YES
7                 ApprovedChangesAmount   decimal         YES
8                            AssignedTo  nvarchar         YES
9                             AwardDate  datetime         YES
10                     ClientCompany_Id    bigint         YES
11             ClientCompanyLocation_Id    bigint         YES
12               ClientCompanyOffice_Id    bigint         YES
13                     ClientContact_Id    bigint         YES
14           Contra

C:\Users\AWright\AppData\Local\Temp\ipykernel_34784\3188530906.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  columns = pd.read_sql(col_query, conn)
C:\Users\AWright\AppData\Local\Temp\ipykernel_34784\3188530906.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sample = pd.read_sql(sample_query, conn)


,Id,DomainPartitionId,PartitionId,CreatedDateTime,ModifiedDateTime,EntityLinkUrl,Addenda,ApprovedChangesAmount,AssignedTo,AwardDate,...,CurrencyCode,CurrencyRateId,CurrencyRateTypeId,CurrencyRateToDomain,CurrencyRateToDocument,CurrencyRateLastModifiedDateTime,LinkChangeOrderItemsToContractItems,CreatedBy_Id,CostUnitEntryType,OverbillingLimits
0,5024152,5023805,5023805,2020-06-03 15:47:37.017,2022-08-04 19:00:25.147,https://demo01launch.kahua.com/link/SCD0z4CA998,None,3500.0,None,NaT,...,USD,714,617,1.0,1.0,2020-06-03 15:47:37.000,None,5005854,TotalValue,None
1,5024158,5023805,5023805,2020-06-03 15:50:54.990,2022-08-04 19:00:25.147,https://demo01launch.kahua.com/link/SCD0z4CA99E,None,0.0,None,NaT,...,USD,714,617,1.0,1.0,2020-06-03 15:50:54.960,None,5005854,TotalValue,None
2,5024245,5023805,5023805,2020-06-03 16:30:52.767,2022-08-04 19:00:25.147,https://demo01launch.kahua.com/link/SCD0z4CA9F5,None,0.0,None,NaT,...,USD,714,617,1.0,1.0,2020-06-03 16:30:52.753,None,5005854,TotalValue,None
3,5028065,5024551,5024551,2020-06-03 21:35:50.250,2023-06-19 14:03:27.187,https://demo01launch.kahua.com/link/SCD0z4CB8E1,None,127845.0,None,2020-04-01,...,USD,714,617,1.0,1.0,2020-06-03 21:35:50.220,None,5005854,TotalValue,None
4,5212019,5024551,5024551,2020-07-07 17:10:49.073,2023-02-01 15:14:42.773,https://demo01launch.kahua.com/link/SCD0z4F8773,None,9000.0,None,NaT,...,USD,714,617,1.0,1.0,2020-07-07 17:10:49.040,None,5005854,TotalValue,None


In [22]:
# Run a custom query
def run_query(sql, limit=100):
    """Execute a SQL query and return results as DataFrame"""
    df = pd.read_sql(sql, conn)
    print(f"Returned {len(df)} rows, {len(df.columns)} columns")
    return df

# Example: Get all contracts with key fields
contracts_df = run_query("""
    SELECT TOP 50 * FROM [kahua_Contract_Contract]
""")
contracts_df

Returned 50 rows, 95 columns


C:\Users\AWright\AppData\Local\Temp\ipykernel_34784\2411187552.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


,Id,DomainPartitionId,PartitionId,CreatedDateTime,ModifiedDateTime,EntityLinkUrl,Addenda,ApprovedChangesAmount,AssignedTo,AwardDate,...,CurrencyCode,CurrencyRateId,CurrencyRateTypeId,CurrencyRateToDomain,CurrencyRateToDocument,CurrencyRateLastModifiedDateTime,LinkChangeOrderItemsToContractItems,CreatedBy_Id,CostUnitEntryType,OverbillingLimits
0,5024152,5023805,5023805,2020-06-03 15:47:37.017,2022-08-04 19:00:25.147,https://demo01launch.kahua.com/link/SCD0z4CA998,None,3500.0,None,NaT,...,USD,714,617,1.0,1.0,2020-06-03 15:47:37.000,None,5005854.0,TotalValue,None
1,5024158,5023805,5023805,2020-06-03 15:50:54.990,2022-08-04 19:00:25.147,https://demo01launch.kahua.com/link/SCD0z4CA99E,None,0.0,None,NaT,...,USD,714,617,1.0,1.0,2020-06-03 15:50:54.960,None,5005854.0,TotalValue,None
2,5024245,5023805,5023805,2020-06-03 16:30:52.767,2022-08-04 19:00:25.147,https://demo01launch.kahua.com/link/SCD0z4CA9F5,None,0.0,None,NaT,...,USD,714,617,1.0,1.0,2020-06-03 16:30:52.753,None,5005854.0,TotalValue,None
3,5028065,5024551,5024551,2020-06-03 21:35:50.250,2023-06-19 14:03:27.187,https://demo01launch.kahua.com/link/SCD0z4CB8E1,None,127845.0,None,2020-04-01,...,USD,714,617,1.0,1.0,2020-06-03 21:35:50.220,None,5005854.0,TotalValue,None
4,5212019,5024551,5024551,2020-07-07 17:10:49.073,2023-02-01 15:14:42.773,https://demo01launch.kahua.com/link/SCD0z4F8773,None,9000.0,None,NaT,...,USD,714,617,1.0,1.0,2020-07-07 17:10:49.040,None,5005854.0,TotalValue,None
5,5373562,5370105,5370105,2020-07-17 13:44:40.183,2022-11-30 18:16:12.433,https://demo01launch.kahua.com/link/SCD0z51FE7A,None,0.0,None,NaT,...,USD,714,617,1.0,1.0,2020-07-17 13:44:40.153,None,5005854.0,TotalValue,None
6,5373592,5370105,5370105,2020-07-17 13:46:55.327,2022-11-09 15:58:39.373,https://demo01launch.kahua.com/link/SCD0z51FE98,None,21000.0,Pat The PM - Summit,NaT,...,USD,714,617,1.0,1.0,2020-07-17 13:46:55.217,None,5005854.0,TotalValue,None
7,5373635,5370105,5370105,2020-07-17 13:49:19.920,2023-02-07 21:16:23.390,https://demo01launch.kahua.com/link/SCD0z51FEC3,None,0.0,None,NaT,...,USD,714,617,1.0,1.0,2020-07-17 13:49:19.890,None,5005854.0,TotalValue,None
8,5376878,5370105,5370105,2020-07-20 13:29:37.593,2022-11-30 18:16:09.780,https://demo01launch.kahua.com/link/SCD0z520B6E,None,0.0,None,NaT,...,USD,714,617,1.0,1.0,2020-07-20 13:29:37.560,None,5373886.0,TotalValue,None
9,5397624,5022361,5022361,2020-07-26 21:41:26.920,2022-08-04 19:00:25.147,https://demo01launch.kahua.com/link/SCD0z525C78,None,143200.0,None,NaT,...,USD,714,617,1.0,1.0,2020-07-26 21:36:18.437,None,5007445.0,UnitPrice,None


In [ ]:
# Search for XML tokens, portable views, and related tables
print("🔍 Searching for XML/Portable View related tables:\n")

for keyword in ['xml', 'token', 'portable', 'view', 'template', 'report']:
    results = search_tables(keyword)
    if len(results) > 0:
        print(results['TABLE_NAME'].tolist())
    print()

🔍 Searching for XML/Portable View related tables:

Found 0 tables matching 'xml':

Found 0 tables matching 'token':

Found 0 tables matching 'portable':

Found 62 tables matching 'view':
['centrenium_PhasegateItem_PhasegateItemReviewApprovalStep', 'centrenium_PhasegateItem_PhasegateItem_CompletedSetupReviews', 'centrenium_PhasegateItem_PhasegateItem_CompletedCompletionReviews', 'kahua_PurchaseOrderChangeOrder_PurchaseOrderChangeOrder_ReviewResults', 'centrenium_PhasegateStage_PhasegateStageSetupReviewApproval', 'centrenium_PhasegateItem_PhasegateItem_SetupReviewItems', 'centrenium_PhasegateStage_PhasegateStageSetupReviewApproval_Steps', 'kahua_Issue_Issue_ReviewResults', 'kahua_AEC_Invoice_Invoice_ReviewResults', 'centrenium_PhasegateItem_PhasegateItem_CompletionReviewItems', 'kahua_AEC_SubmittalItem_SubmittalItem_OfficialReviewer', 'kahua_AEC_SubmittalPackage_SubmittalPackage_OfficialReviewer', 'centrenium_PhasegateStage_PhasegateStageReviewApprovalStep', 'kahua_AEC_SubmittalItem_Subm

: 